# 03 CNN

In [ ]:
import config
from utils import data_model
from utils.model import Model

import pandas as pd
import numpy as np
import tensorflow as tf

## Load data

In [ ]:
data, imgs_left, imgs_right = data_model.load(
    config.PATH_DATA, config.PATH_IMGS_LEFT, config.PATH_IMGS_RIGHT
)

In [ ]:
data.head()

## Split data

In [ ]:
(
    (train_data, train_imgs_left, train_imgs_right),
    (validation_data, validation_imgs_left, validation_imgs_right),
    (test_data, test_imgs_left, test_imgs_right)
) = data_model.split(
    data, imgs_left, imgs_right,
    train_size=0.95,
    validation_size=0.95,
    random_state=42
)

In [ ]:
print("Train length: {}".format(len(train_data)))
print("Validation length: {}".format(len(validation_data)))
print("Test length: {}".format(len(test_data)))

## Model

### Architecture

In [ ]:
def get_model(features, left_imgs, right_imgs, keep_prob):
    new_shape = np.array([-1, 20, 30, 1])
    with tf.variable_scope('model'):
        # Left Eye Img
        l_input = tf.reshape(left_imgs, new_shape)
        # 20x30x1
        cnn_l_01 = tf.layers.conv2d(
            inputs=l_input, 
            filters=32, 
            kernel_size=5, 
            strides=1,
            padding="SAME"
        )
        # 20x30x32
        cnn_l_02 = tf.layers.conv2d(
            inputs=cnn_l_01, 
            filters=64, 
            kernel_size=5, 
            strides=2,
            padding="SAME"
        )
        # 10x15x64
        cnn_l_03 = tf.layers.conv2d(
            inputs=cnn_l_02, 
            filters=128, 
            kernel_size=3, 
            strides=2,
            padding="VALID"
        ) 
        # 4x7x128
        
        # Right Eye Img  
        r_input = tf.reshape(right_imgs, new_shape)      
        # 20x30x1
        cnn_r_01 = tf.layers.conv2d(
            inputs=r_input, 
            filters=32, 
            kernel_size=5, 
            strides=1,
            padding="SAME"
        )
        # 20x30x32
        cnn_r_02 = tf.layers.conv2d(
            inputs=cnn_r_01, 
            filters=64, 
            kernel_size=5, 
            strides=2,
            padding="SAME"
        )
        # 10x15x64
        cnn_r_03 = tf.layers.conv2d(
            inputs=cnn_r_02, 
            filters=128, 
            kernel_size=3, 
            strides=2,
            padding="VALID"
        )
        # 4x7x128
        
        # Flatten convs, concat & dense        
        left_flat = tf.contrib.layers.flatten (cnn_l_03)
        right_flat =  tf.contrib.layers.flatten (cnn_r_03)
        img_concat = tf.concat(
            values=[left_flat, right_flat],
            axis=1
        )
        img_dense = tf.layers.dense(
            inputs=img_concat,
            units=128,
            activation=tf.nn.relu,
        )
        img_dropout = tf.nn.dropout(
            x=img_dense,
            keep_prob=keep_prob
        )
        
        # Concat imgs with features, dense x 2 and output
        global_concat = tf.concat(
            values=[features, img_dropout],
            axis=1
        )
        global_dense01 = tf.layers.dense(global_concat, 128, activation=tf.nn.relu)
        global_dropout01 = tf.nn.dropout(
            x=global_dense01,
            keep_prob=keep_prob
        )
        global_dense02 = tf.layers.dense(global_dropout01, 64, activation=tf.nn.relu)
        global_dropout02 = tf.nn.dropout(
            x=global_dense02,
            keep_prob=keep_prob
        )
        global_dense03 = tf.layers.dense(global_dropout01, 2, activation=None)
        
        return global_dense03

### Parameters

In [ ]:
MODEL_NAME = 'cnn-05'

EPOCHS = 25
BATCH_SIZE = 256
LEARNING_RATE = 0.003
KEEP_PROB = 1.0

## Runs

In [ ]:
model = Model(MODEL_NAME, get_model)

### Train

In [ ]:
model.train(
    train_data, train_imgs_left, train_imgs_right,
    validation_data, validation_imgs_left, validation_imgs_right,
    BATCH_SIZE, EPOCHS, LEARNING_RATE, KEEP_PROB
)

### Test

In [ ]:
model_test = Model(MODEL_NAME, saved_model=MODEL_NAME+".final")

In [ ]:
model_test.test(
    test_data, test_imgs_left, test_imgs_right
)

### Results


| Name | Epochs | Batch Size | Learning rate  | Keep_prob | Train | Validation | Test |
|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|
| cnn-01 | 10 | 512 | 0.005 | 0.8 | 0.36621132493019104 | 0.36725661158561707 | 0.35972196 |
| cnn-02 | 10 | 128 | 0.005 | 0.8 | 0.5809355974197388 | 0.5789308547973633 | 0.57856214 |
| cnn-03 | 10 | 256 | 0.005 | 0.8 | 0.330032616853714 | 0.3361927270889282 | 0.33274087 |
| cnn-04 | 40 | 256 | 0.001 | 0.9 | 0.12016592919826508 | 0.20632070302963257 | 0.20110765 |
| cnn-05 | 25 | 256 | 0.003 | 1.0 | 0.17252811789512634 | 0.23917752504348755 | 0.20110765 |
